#### <u>Predictive models and reasoning with explanations</u>:

In this notebook, we will implement the code pertaining to Ryma Boumazouza's approach for enumerating sufficient reasons and counterfactual explanations behind a decision.

In [1]:
import sympy as sp
from sympy import satisfiable
from sympy.logic import to_cnf

import subprocess
import tools.minihit

from functools import reduce

##### <u>1. Computing suficient reasons and contrfactuals explanations</u>:
In this section, given a classifier and an instance, we compute the sufficient reasons and counterfactual explanations behind the classifier's decision regarding this instance.

In [2]:
# Definition of some auxiliary functions that allow us to obtain the WCNF file.

def count_clauses(formula)-> int:
    """
    Returns the number of clauses of the formula.
    """
    if isinstance(formula, sp.And):
        return len(formula.args)
    elif isinstance(formula, sp.Or):
        return 1
    else:
        return 1


def map_symbols_to_integers(symbols):
    """
    Maps each symbol to a unique integer starting from 1.
    """

    symbol_map = {symbol: i + 1 for i, symbol in enumerate(symbols)}
    return symbol_map

In [3]:
def clf_ins(clf, instance)-> tuple:
    """
    Returns the complementary form of `clf` if `instance` implies `clf`,
    otherwise returns `clf` and `instance`.

    Parameters:
    ----------
    clf : sympy expression
        The classifier or formula to be checked.
        
    instance : sympy expression
        The instance or condition to be checked against `clf`.

    Returns:
    -------
    tuple
        A tuple containing the prediction of the classifier, the modified `clf` and `instance` based on the implication check.
    """

    if(not satisfiable(sp.And(sp.Not(clf), instance))):
        return 1, to_cnf(sp.Not(clf)), instance
    return 0, clf, instance

In [24]:
def to_dimacs(clf, instance, symbol_map, top=2)-> list[str]:
    """
    Converts logical formulas clf and instance into DIMACS format clauses using
    a symbol mapping symbol_map.

    Parameters:
    ----------
    clf : sympy expression
        The classifier or main logical formula to be converted.

    instance : sympy expression
        The instance or additional logical condition to be converted.

    symbol_map : dict
        A dictionary mapping sympy symbols or literals to integer identifiers used
        in the DIMACS format.

    top : int, optional
        Integer identifier to indicate the type of clause (e.g., hard or soft).
        Defaults to 2 for hard clauses.

    Returns:
    -------
    list
        A list of strings representing the clauses in DIMACS format.
    """
    
    # Hard clauses
    clauses = []
    if isinstance(clf, sp.And):
        for clause in clf.args:
            if isinstance(clause, sp.Or):
                clauses.append(str(top) + ' ' + ' '.join([str(symbol_map[lit]) if not lit.is_Not else f"-{symbol_map[lit.args[0]]}" for lit in clause.args]))
            else:
                clauses.append(str(top) + ' ' + str(symbol_map[clause]) if not clause.is_Not else f"{top} -{symbol_map[clause.args[0]]}")

    elif isinstance(clf, sp.Or):
        clauses.append(str(top) + ' ' + ' '.join([str(symbol_map[lit]) if not lit.is_Not else f"-{symbol_map[lit.args[0]]}" for lit in clf.args]))
        
    else:
        clauses.append(str(top) + ' ' + str(symbol_map[clf.args[0]]) if not clf.is_Not else f"{top} -{symbol_map[clf.args[0]]}")

    # Soft clauses
    if isinstance(instance, sp.And):
        for clause in instance.args:
            if isinstance(clause, sp.Or):
                clauses.append('1 ' + ' '.join([str(symbol_map[lit]) if not lit.is_Not else f"-{symbol_map[lit.args[0]]}" for lit in clause.args]))
            else:
                clauses.append('1 ' + str(symbol_map[clause]) if not clause.is_Not else f"1 -{symbol_map[clause.args[0]]}")

    elif isinstance(instance, sp.Or):
        clauses.append('1 ' + ' '.join([str(symbol_map[lit]) if not lit.is_Not else f"-{symbol_map[lit.args[0]]}" for lit in clf.args]))
        
    else:
        clauses.append('1 ' + str(symbol_map[instance.args[0]]) if not instance.is_Not else f"1 -{symbol_map[instance.args[0]]}")
    
    return clauses

In [5]:
def formula_to_file(fileName: str, formula: list[str], vars: int, claus: int, top= 2)-> None:
    """
    Write the formula 'formula' in a file titled 'fileName' in a DIMACS format.

    Parameters:
    ----------
    fileName (str): The title of the file produced.
    formula (list[list[int]]): The formula to be converted.
    vars (int): The number of the variables in the formula 'formula'.
    claus (int): The number of the clauses in the formula 'formula'.
    top (int): It is an integer such that if the weight of a clause is equal to it, the clause is considered as hard. 
    """

    with open(fileName, 'w') as f:
        f.write(f'p wcnf {vars} {claus} {top}\n')
        for clause in formula:
            f.write(f'{clause} 0\n')
        f.close()

In [6]:
def run(command: str):
    """
    Execute a terminal command and return the output. 
    """

    result = subprocess.run(command, shell= True, stdout= subprocess.PIPE, stderr= subprocess.PIPE, text= True)
    if result.returncode == 0:
        return result.stdout
    else:
        raise Exception(f"Command failed with error: {result.stderr}")

In [7]:
def mcs_s(fileName: str)-> list[set[int]]:
    """
    Returns a set of minimal correction subsets generated by enumELSRMRCache.

    Parameters:
    ----------
    fileName (str): The title of the file produced by enumELSRMRCache containing minimal correction subsets.
    """

    mincos = []
    with open (fileName, 'r') as f:
        line = f.readline()
        while(line != ''):
            if (line[0] == 'v'):
                mincos.append(set((line[2:]).split(' ')[:-1]))
            line = f.readline()
        f.close()

    return mincos

In [8]:
def mhs_s(mcs: list[set[int]])-> list[set[int]]:
    """
    Returns a set of minimal hitting sets of MCS (Minimal Correction Subsets).

    Parameters:
    ----------
    mcs (list[set[int]]): List of sets representing the Minimal Correction Subsets (MCS).

    Returns:
    -------
    set[set[int]]: A set of sets representing the Minimal Hitting Sets (MHS).
    """

    mhs = tools.minihit.HsDag(mcs)
    mhs.solve(prune= True, sort= False)
    return list(mhs.generate_minimal_hitting_sets())    

In [9]:
def xPred(clf, instance)-> tuple[int, list[set[int]], list[set[int]]]:
    """
    Compute the prediction, the sufficient reasons, and the counterfactual explanations for the decision taken by the classifier 'clf' on the
    given 'instance'.
    
    Parameters:
    ----------
    clf : sp.Expr
        The classifier formula.
    instance : sp.Expr
        The instance formula.
    
    Returns:
    -------
    tuple[int, list[set[int]], list[set[int]]]
        A tuple containing the prediction, a list of sufficient reasons (minimal hitting sets), and a list of counterfactual explanations (minimal correction subsets).
    """

    # Mapping symbols to integers to write the formula in DIMACS format.
    symbols = sorted(clf.free_symbols, key=lambda x: x.name)
    symbol_map = map_symbols_to_integers(symbols)

    # Determine prediction and possibly modify clf and instance for unsatisfiable formulas.
    pred, clf, instance = clf_ins(clf, instance)
    
    # Write the formula in DIMACS format.
    formula = to_dimacs(clf, instance, symbol_map)
    formula_to_file('xPred.wcnf', formula, len(symbols), len(formula))

    # Compress the formula file and run the external tool.
    run('gzip xPred.wcnf')
    run('./tools/enumELSRMRCache -verb=1 xPred.wcnf.gz > xPred.txt')

    # Compute the minimal correction subsets (MCS) and minimal hitting sets (MHS).
    mincos = mcs_s('xPred.txt')
    minhs = mhs_s(mincos)

    # Clean up temporary files.
    run('rm xPred.wcnf.gz xPred.txt')

    # Helper function to map MCS/MHS elements to their corresponding values in the formula.
    def aux(x: set[str], formula=formula)-> set[int]:
        result = set()
        for y in x:
            index = int(y) - 1
            if 0 <= index < len(formula):
                value = formula[index][1:4].strip()  # Remove spaces
                result.add(int(value))
        return result

    # Apply the helper function to the lists of MCS and MHS.
    mincos = list(map(lambda x: aux(x, formula), mincos))
    minhs = list(map(lambda x: aux(x, formula), minhs))

    return pred, minhs, mincos


##### <u>2. Computing Queries</u>:

We can now compute various queries using the previous code, specially:
> **Necessary property.**<br>
> **Necessary Reason.**<br>
> **Because Statments.**<br>
> **Decision Bias.**<br>

In [10]:
def necessary_property(srx: list[set[int]])-> set[int]:
    """
    Returns the set of literlas which are necessary for decision.

    Parameters:
    ----------
    srx (list[set[int]]): A list of sufficient reasons.
    """ 
    return reduce(lambda x,y: x & y, srx)


In [11]:
def necessary_reason(srx: list[set[int]])-> set[int]:
    """
    Returns the necessary reason for the decision if it exists.

    Parameters:
    ----------
    srx (list[set[int]]): A list of sufficient reasons.
    """
    return srx[0] if len(srx) == 1 else set()

In [12]:
def because_t(srx: list[set[int]], t: set[int])-> bool:
    """
    Decide whether the decision was made because of the given property.

    Parameters:
    ----------
    srx (list[set[int]]): 
    """
    return t == necessary_reason(srx)

In [13]:
def decision_bias(srx: list[set[int]], unprotected_vars: list[bool])-> set[int]:
    """
    Decide whether the decision Δ(a) is biased.

    Parameters:
    ----------
    srx (list[set[int]]): A list of sufficient reasons.
    unprotected_vars (list[bool]): A list of boolean values where each value indicates whether the corresponding variable is unprotected.

    Returns:
    -------
    bool: True if the decision is biased, False otherwise.
    """
    nec = necessary_property(srx)
    return any(x for x in necessary_property(srx) if unprotected_vars[abs(x) - 1] == 0)

##### <u>3. Case study</u>:
In this final section, we will validate our previous code -developped in sections 1 and 2 using a case study proposed by Adnan Darwiche in his article 'On the (Complete) Reasons Behind Decisions.'

In [15]:
# Building the logic classifier (Admission classifier):
clf = sp.simplify_logic(sp.sympify('(C & ((F & (G | W)) | (~F & R))) | (G & R & W)'))

# Defining the applicants:
scott = sp.sympify('C & ~F & G & R & W')
robin = sp.sympify('C & F & G & R & W')
april = sp.sympify('C & F & G & ~R & W')

# Building the predicition, sufficient reasons, conterfacuals explanations sets.
ps, sfs, cfs = xPred(clf, scott)
pr, sfr, cfr = xPred(clf, robin)
pa, sfa, cfa = xPred(clf, april)

**<u>Scott</u>**:<br><br>
According to Darwiche's article, the decision regarding the applicant Scott is biased. The only protected feature is R. Let's verify if our code confirms this.

In [16]:
decision_bias(sfs, [1, 1, 1, 0, 1])

True

We can also verify if our code arrives at the same sufficient reasons as those in the article, namely:
> $(C, G, R); (C, R, W); (C, R, -F); (G, R, W)$<br>
> $(1, 3, 4); (1, 4, 5); (1, 4, -2); (3, 4, 5)$

In [17]:
sfs

[{-2, 1, 4}, {1, 4, 5}, {1, 3, 4}, {3, 4, 5}]

As stated in the article, if we flip the protected characteristic R to -R, the decision will flip with the complete reason being -F, -R so Scott would be denied admission *because* he is not a first time applicant and does not come from a rich hometown.

In [18]:
scottf = sp.sympify('C & ~F & G & ~R & W')
psf, sfsf, cfsf = xPred(clf, scottf)

print(f'The complete reason behind the decision is {sfsf}')
b = because_t(sfsf, {-2, -4})
print(f'Scott would be denied admission because he is not a first time applicant and does not come from a rich hometown = {b}')

The complete reason behind the decision is [{-4, -2}]
Scott would be denied admission because he is not a first time applicant and does not come from a rich hometown = True


**<u>Robin</u>**:<br><br>
According to Darwiche's article, the decision regarding the applicant Robin is not biased. The only protected feature is R. Let's verify if our code confirms this.

In [19]:
decision_bias(sfr, [1, 1, 1, 0, 1])

False

We can also verify if our code arrives at the same sufficient reasons as those in the article, namely:
> $(C, F, G); (C, F, W); (C, R, G); (C, R, W); (G, R, W)$<br>
> $(1, 2, 3); (1, 2, 5); (1, 4, 3); (1, 4, 5); (3, 4, 5)$

In [20]:
sfr

[{1, 3, 4}, {1, 2, 3}, {1, 4, 5}, {1, 2, 5}, {3, 4, 5}]

**<u>April</u>**:<br><br>
According to Darwiche's article, the decision regarding the applicant April is not biased. The only protected feature is R. Let's verify if our code confirms this.

In [21]:
decision_bias(sfa, [1, 1, 1, 0, 1])

False

Moreover, $C, F$ are all the necessary characteristics for this decision (i.e., the necessary property). Let's confirm that.

In [22]:
necessary_property(sfa)

{1, 2}

As stated in the article, the decision on April would stick *even if* she were not to have work experience $(-W)$ because she passed the entrance exam $(C)$, has a good GPA $(G)$ and is a first time applicant $(F)$.

In [23]:
aprilf = sp.sympify('C & F & G & ~R & ~W')
paf, sfaf, cfaf = xPred(clf, aprilf)

b1 = (paf == pa)
b2 = because_t(sfaf, {1, 2, 3})

print(f'The decision would stick even if (-W): {b1}, because C, G and F: {b2}')

The decision would stick even if (-W): True, because C, G and F: True
